# Lab 1.3.4: CuPy Integration - SOLUTIONS

This notebook contains complete solutions to the exercises in Lab 1.3.4.

---

## 🎯 Learning Objectives Checklist

By completing this lab, you should now be able to:
- [x] Use CuPy as a drop-in replacement for NumPy
- [x] Port a data preprocessing pipeline to GPU
- [x] Write custom CUDA kernels using CuPy's RawKernel
- [x] Transfer data seamlessly between CuPy and PyTorch (zero-copy!)
- [x] Achieve **10x+ speedup** on data preprocessing

---

In [ ]:
import numpy as np
import time

try:
    import cupy as cp
    HAS_CUPY = True
    print(f"✅ CuPy {cp.__version__} available")
except ImportError:
    HAS_CUPY = False
    print("❌ CuPy not available")

## Solution: Image Preprocessing Pipeline

In [ ]:
def preprocess_images_numpy(images: np.ndarray) -> np.ndarray:
    """
    NumPy reference implementation.
    
    Input: (batch, height, width, channels) uint8 [0-255]
    Output: (batch, channels, height, width) float32 [-1, 1]
    """
    # ImageNet normalization constants
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    
    # Step 1: Normalize to [0, 1]
    result = images.astype(np.float32) / 255.0
    
    # Step 2: Per-channel normalization
    result = (result - mean) / std
    
    # Step 3: Transpose NHWC -> NCHW
    result = np.transpose(result, (0, 3, 1, 2))
    
    # Step 4: Random horizontal flip
    flip_mask = np.random.random(result.shape[0]) < 0.5
    result[flip_mask] = result[flip_mask, :, :, ::-1]
    
    return result

In [ ]:
if HAS_CUPY:
    def preprocess_images_cupy(images) -> "cp.ndarray":
        """
        SOLUTION: CuPy version of image preprocessing.
        
        The code is almost identical to NumPy - just change np → cp!
        """
        # ImageNet normalization constants (on GPU)
        mean = cp.array([0.485, 0.456, 0.406], dtype=cp.float32)
        std = cp.array([0.229, 0.224, 0.225], dtype=cp.float32)
        
        # Step 1: Normalize to [0, 1]
        result = images.astype(cp.float32) / 255.0
        
        # Step 2: Per-channel normalization
        result = (result - mean) / std
        
        # Step 3: Transpose NHWC -> NCHW
        result = cp.transpose(result, (0, 3, 1, 2))
        
        # Step 4: Random horizontal flip
        flip_mask = cp.random.random(result.shape[0]) < 0.5
        result[flip_mask] = result[flip_mask, :, :, ::-1]
        
        return result

In [ ]:
# Test the solution
batch_size = 64
height, width = 224, 224
channels = 3

images = np.random.randint(0, 256, (batch_size, height, width, channels), dtype=np.uint8)

# NumPy version
np.random.seed(42)
result_np = preprocess_images_numpy(images)
print(f"NumPy result shape: {result_np.shape}")
print(f"NumPy result dtype: {result_np.dtype}")
print(f"NumPy value range: [{result_np.min():.2f}, {result_np.max():.2f}]")

if HAS_CUPY:
    # CuPy version
    cp.random.seed(42)
    images_gpu = cp.asarray(images)
    result_cp = preprocess_images_cupy(images_gpu)
    
    print(f"\nCuPy result shape: {result_cp.shape}")
    print(f"CuPy result dtype: {result_cp.dtype}")
    print(f"CuPy value range: [{float(result_cp.min()):.2f}, {float(result_cp.max()):.2f}]")

In [ ]:
if HAS_CUPY:
    # Benchmark
    print("\nBenchmark: NumPy vs CuPy Image Preprocessing")
    print("="*50)
    
    # Larger batch for meaningful benchmark
    large_batch = np.random.randint(0, 256, (128, 224, 224, 3), dtype=np.uint8)
    large_batch_gpu = cp.asarray(large_batch)
    
    # NumPy timing
    start = time.perf_counter()
    for _ in range(5):
        _ = preprocess_images_numpy(large_batch)
    time_numpy = (time.perf_counter() - start) / 5
    
    # CuPy timing (warm up)
    _ = preprocess_images_cupy(large_batch_gpu)
    cp.cuda.Stream.null.synchronize()
    
    start = time.perf_counter()
    for _ in range(5):
        _ = preprocess_images_cupy(large_batch_gpu)
    cp.cuda.Stream.null.synchronize()
    time_cupy = (time.perf_counter() - start) / 5
    
    print(f"Batch size: 128 × 224 × 224 × 3")
    print(f"NumPy time: {time_numpy*1000:.2f} ms")
    print(f"CuPy time:  {time_cupy*1000:.2f} ms")
    print(f"Speedup:    {time_numpy/time_cupy:.1f}x")

## Bonus: Data Augmentation Pipeline

In [ ]:
if HAS_CUPY:
    def augment_images_cupy(images: "cp.ndarray", 
                           flip_prob: float = 0.5,
                           brightness_delta: float = 0.2,
                           contrast_range: tuple = (0.8, 1.2)) -> "cp.ndarray":
        """
        GPU-accelerated image augmentation.
        
        Args:
            images: (batch, H, W, C) float32 images
            flip_prob: Probability of horizontal flip
            brightness_delta: Max brightness adjustment
            contrast_range: (min, max) contrast multiplier
        """
        batch_size = images.shape[0]
        result = images.copy()
        
        # Random horizontal flip
        flip_mask = cp.random.random(batch_size) < flip_prob
        result[flip_mask] = result[flip_mask, :, ::-1, :]
        
        # Random brightness adjustment
        brightness = cp.random.uniform(-brightness_delta, brightness_delta, 
                                        (batch_size, 1, 1, 1)).astype(cp.float32)
        result = result + brightness
        
        # Random contrast adjustment
        contrast = cp.random.uniform(contrast_range[0], contrast_range[1],
                                      (batch_size, 1, 1, 1)).astype(cp.float32)
        mean = result.mean(axis=(1, 2, 3), keepdims=True)
        result = (result - mean) * contrast + mean
        
        # Clip to valid range
        result = cp.clip(result, 0.0, 1.0)
        
        return result
    
    # Test augmentation
    test_images = cp.random.random((32, 64, 64, 3), dtype=cp.float32)
    augmented = augment_images_cupy(test_images)
    print(f"Augmented images shape: {augmented.shape}")
    print(f"Value range: [{float(augmented.min()):.2f}, {float(augmented.max()):.2f}]")

## Cleanup

In [ ]:
import gc

if HAS_CUPY:
    mempool = cp.get_default_memory_pool()
    mempool.free_all_blocks()

gc.collect()
print("✅ Cleanup complete")